
# 🔄 Swing Trading Bot — End‑to‑End Pipeline (NSE/BSE, India)

**Version:** 2025-09-28  
**Timezone:** Asia/Kolkata (IST)  

This notebook implements a **stage‑based, modular pipeline** for **swing trading** on the Indian stock market (NSE/BSE).  
Each stage is self‑contained and documented.

> ⚠️ **Disclaimer:** Educational use only. Validate and trade responsibly.



## Stage 0 — Environment & Dependencies


In [13]:

# Optional installs (uncomment to run locally)
# %pip install -U yfinance pandas numpy matplotlib scikit-learn python-dateutil pytz
# %pip install -U lightgbm  # optional for ML filter



## Global Imports & Configuration


In [2]:

import os, math, time, json, warnings
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
import pytz

try:
    import yfinance as yf
except Exception as e:
    print("yfinance not available. Install with `%pip install yfinance` if you plan to download data.")

# ML (optional)
try:
    from sklearn.model_selection import TimeSeriesSplit
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
except Exception:
    pass

pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 200)

IST = pytz.timezone("Asia/Kolkata")

CONFIG: Dict[str, Any] = {
    "DATA": {
        "tickers": ["HDFCBANK.NS", "RELIANCE.NS", "TCS.NS", "INFY.NS", "ICICIBANK.NS"],
        "start_date": "2018-01-01",
        "end_date": None,  # None = today
        "interval": "1d",
        "cache_dir": "cache/data",
        "force_refresh": False
    },
    "FEATURES": {
        "use_indicators": True,
        "indicators": ["SMA20", "SMA50", "EMA20", "RSI14", "MACD_12_26_9", "BBANDS_20_2", "ATR14", "ADX14"],
        "include_patterns": True,
        "include_stats": True
    },
    "SCREEN": {
        "min_avg_volume": 200000.0,
        "price_min": 50.0,
        "price_max": 5000.0,
        "trend_rule": "SMA20_gt_SMA50",
        "atr_pct_range": (0.5, 4.5)
    },
    "SIGNAL": {
        "rules": [
            "SMA20_gt_SMA50",
            "RSI_gt_55",
            "MACD_hist_gt_0",
            "Close_gt_BB_mid"
        ],
        "ml_filter_enabled": False,
        "ml_prob_threshold": 0.60,
        "label_horizon_days": 10,
        "label_thr_pct": 4.0
    },
    "RISK": {
        "initial_capital": 500000.0,
        "risk_per_trade_pct": 1.0,
        "atr_stop_mult": 1.5,
        "atr_trail_mult": 1.5,
        "max_open_positions": 8,
        "max_hold_days": 12
    },
    "COSTS": {
        "brokerage_per_order": 20.0,
        "slippage_bps": 5.0,
        "stt_equity_deliv_bps": 100.0,
        "exchange_txn_bps": 3.25,
        "clearing_bps": 0.0,
        "stamp_duty_bps_buy": 15.0,
        "gst_pct_on_brokerage": 18.0
    },
    "OUTPUTS": {
        "base_dir": "outputs",
        "save_equity_plot": True,
        "save_csv": True
    }
}

if CONFIG["DATA"]["end_date"] is None:
    CONFIG["DATA"]["end_date"] = datetime.now(IST).strftime("%Y-%m-%d")

def get_outdir() -> Path:
    d = datetime.now(IST).strftime("%Y-%m-%d")
    p = Path(CONFIG["OUTPUTS"]["base_dir"]) / d
    p.mkdir(parents=True, exist_ok=True)
    return p

get_outdir()
print("✅ CONFIG ready")


✅ CONFIG ready



## Stage 1 — Data Ingestion


In [3]:

from pathlib import Path

def ensure_dir(path: str | Path) -> Path:
    p = Path(path)
    p.mkdir(parents=True, exist_ok=True)
    return p

def normalize_ohlcv(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    rename_map = {"Open":"open","High":"high","Low":"low","Close":"close","Adj Close":"adj_close","Volume":"volume"}
    df.rename(columns=rename_map, inplace=True)
    if not isinstance(df.index, pd.DatetimeIndex):
        if "Date" in df.columns:
            df.set_index(pd.to_datetime(df["Date"]), inplace=True)
            df.drop(columns=["Date"], inplace=True, errors="ignore")
        else:
            raise ValueError("Missing DatetimeIndex or Date column")
    df.index = df.index.tz_localize("UTC").tz_convert(IST) if df.index.tz is None else df.index.tz_convert(IST)
    df = df.sort_index()
    return df[["open","high","low","close","adj_close","volume"]].dropna(how="any")

def load_ticker_df(ticker: str, start: str, end: str, interval: str, cache_dir: str, force_refresh: bool=False) -> pd.DataFrame:
    cache_dir = ensure_dir(cache_dir)
    fp = Path(cache_dir) / f"{ticker}_{interval}.parquet"
    if fp.exists() and not force_refresh:
        try:
            return pd.read_parquet(fp)
        except Exception:
            pass
    if "yf" not in globals():
        raise ImportError("Install yfinance or provide cached data.")
    print(f"Downloading {ticker} ...")
    raw = yf.download(ticker, start=start, end=end, interval=interval, auto_adjust=False, progress=False)
    if raw is None or raw.empty:
        raise ValueError(f"No data for {ticker}")
    df = normalize_ohlcv(raw)
    df.to_parquet(fp)
    return df

def load_universe(config: Dict[str, Any]) -> Dict[str, pd.DataFrame]:
    dcfg = config["DATA"]
    out = {}
    for t in dcfg["tickers"]:
        try:
            out[t] = load_ticker_df(t, dcfg["start_date"], dcfg["end_date"], dcfg["interval"], dcfg["cache_dir"], dcfg["force_refresh"])
        except Exception as e:
            print("⚠️", t, e)
    return out

print("✅ Stage 1 ready")


✅ Stage 1 ready



## Stage 2 — Feature Engineering


In [4]:

def SMA(s: pd.Series, n: int) -> pd.Series:
    return s.rolling(n, min_periods=n).mean()

def EMA(s: pd.Series, n: int) -> pd.Series:
    return s.ewm(span=n, adjust=False).mean()

def RSI(close: pd.Series, n: int = 14) -> pd.Series:
    d = close.diff()
    up = np.where(d > 0, d, 0.0)
    dn = np.where(d < 0, -d, 0.0)
    roll_up = pd.Series(up, index=close.index).ewm(alpha=1/n, adjust=False).mean()
    roll_dn = pd.Series(dn, index=close.index).ewm(alpha=1/n, adjust=False).mean()
    rs = roll_up / (roll_dn + 1e-12)
    return 100.0 - (100.0 / (1.0 + rs))

def MACD(close: pd.Series, fast: int=12, slow: int=26, signal: int=9):
    ef, es = EMA(close, fast), EMA(close, slow)
    macd = ef - es
    sig = EMA(macd, signal)
    hist = macd - sig
    return macd, sig, hist

def true_range(df: pd.DataFrame) -> pd.Series:
    pc = df["close"].shift(1)
    return pd.concat([df["high"]-df["low"], (df["high"]-pc).abs(), (df["low"]-pc).abs()], axis=1).max(axis=1)

def ATR(df: pd.DataFrame, n: int=14) -> pd.Series:
    return true_range(df).ewm(alpha=1/n, adjust=False).mean()

def Bollinger(close: pd.Series, n: int=20, nstd: float=2.0):
    mid = close.rolling(n, min_periods=n).mean()
    std = close.rolling(n, min_periods=n).std(ddof=0)
    return mid + nstd*std, mid, mid - nstd*std

def ADX(df: pd.DataFrame, n: int=14) -> pd.Series:
    up = df["high"].diff()
    dn = -df["low"].diff()
    plus_dm = np.where((up > dn) & (up > 0), up, 0.0)
    minus_dm = np.where((dn > up) & (dn > 0), dn, 0.0)
    tr = true_range(df)
    atr = tr.ewm(alpha=1/n, adjust=False).mean()
    plus_di = 100 * (pd.Series(plus_dm, index=df.index).ewm(alpha=1/n, adjust=False).mean() / (atr + 1e-12))
    minus_di = 100 * (pd.Series(minus_dm, index=df.index).ewm(alpha=1/n, adjust=False).mean() / (atr + 1e-12))
    dx = 100 * ((plus_di - minus_di).abs() / ((plus_di + minus_di) + 1e-12))
    return dx.ewm(alpha=1/n, adjust=False).mean()

def bullish_engulfing(df: pd.DataFrame) -> pd.Series:
    o, c = df["open"], df["close"]
    o1, c1 = o.shift(1), c.shift(1)
    return ((c > o) & (c1 < o1) & (c >= o1) & (o <= c1)).astype(int)

def bearish_engulfing(df: pd.DataFrame) -> pd.Series:
    o, c = df["open"], df["close"]
    o1, c1 = o.shift(1), c.shift(1)
    return ((c < o) & (c1 > o1) & (c <= o1) & (o >= c1)).astype(int)

def doji(df: pd.DataFrame, tol: float=0.1) -> pd.Series:
    body = (df["close"] - df["open"]).abs()
    rng = (df["high"] - df["low"]).replace(0, np.nan)
    return (body / rng < tol).fillna(0).astype(int)

def hammer(df: pd.DataFrame) -> pd.Series:
    o, h, l, c = df["open"], df["high"], df["low"], df["close"]
    body = (c - o).abs()
    lower_shadow = o.combine(c, min) - l
    upper_shadow = h - o.combine(c, max)
    return ((lower_shadow >= 2 * body) & (upper_shadow <= body)).astype(int)

def hanging_man(df: pd.DataFrame) -> pd.Series:
    return hammer(df)

def add_features(df: pd.DataFrame, cfg: Dict[str, Any]) -> pd.DataFrame:
    df = df.copy()
    if cfg["FEATURES"]["use_indicators"]:
        if "SMA20" in cfg["FEATURES"]["indicators"]:
            df["SMA20"] = SMA(df["close"], 20)
        if "SMA50" in cfg["FEATURES"]["indicators"]:
            df["SMA50"] = SMA(df["close"], 50)
        if "EMA20" in cfg["FEATURES"]["indicators"]:
            df["EMA20"] = EMA(df["close"], 20)
        if "RSI14" in cfg["FEATURES"]["indicators"]:
            df["RSI14"] = RSI(df["close"], 14)
        if "MACD_12_26_9" in cfg["FEATURES"]["indicators"]:
            macd, macds, mach = MACD(df["close"], 12, 26, 9)
            df["MACD"] = macd
            df["MACD_signal"] = macds
            df["MACD_hist"] = mach
        if "BBANDS_20_2" in cfg["FEATURES"]["indicators"]:
            bb_u, bb_m, bb_l = Bollinger(df["close"], 20, 2.0)
            df["BB_upper"] = bb_u
            df["BB_mid"] = bb_m
            df["BB_lower"] = bb_l
        if "ATR14" in cfg["FEATURES"]["indicators"]:
            df["ATR14"] = ATR(df, 14)
        if "ADX14" in cfg["FEATURES"]["indicators"]:
            df["ADX14"] = ADX(df, 14)

    if cfg["FEATURES"]["include_patterns"]:
        df["bullish_engulfing"] = bullish_engulfing(df)
        df["bearish_engulfing"] = bearish_engulfing(df)
        df["doji"] = doji(df)
        df["hammer"] = hammer(df)
        df["hanging_man"] = hanging_man(df)

    if cfg["FEATURES"]["include_stats"]:
        df["ret_1d"] = df["close"].pct_change()
        df["ret_5d"] = df["close"].pct_change(5)
        df["vol_20"] = df["ret_1d"].rolling(20).std() * np.sqrt(252)
        df["ATR_pct"] = (df["ATR14"] / df["close"]) * 100.0

    return df.dropna().copy()

print("✅ Stage 2 ready")


✅ Stage 2 ready



## Stage 3 — Screening / Filtering


In [5]:

def passes_screen(df: pd.DataFrame, cfg: Dict[str, Any]) -> bool:
    if df is None or len(df) < 100:
        return False
    tail = df.tail(60)
    last = df.iloc[-1]

    if tail["volume"].mean() < cfg["SCREEN"]["min_avg_volume"]:
        return False
    if not (cfg["SCREEN"]["price_min"] <= last["close"] <= cfg["SCREEN"]["price_max"]):
        return False
    if cfg["SCREEN"]["trend_rule"] == "SMA20_gt_SMA50" and not (last.get("SMA20", np.nan) > last.get("SMA50", np.nan)):
        return False
    lo, hi = cfg["SCREEN"]["atr_pct_range"]
    if "ATR_pct" in last.index and not (lo <= last["ATR_pct"] <= hi):
        return False
    return True

print("✅ Stage 3 ready")


✅ Stage 3 ready



## Stage 4 — Signal Generation


In [6]:

def rule_conditions_satisfied(row: pd.Series, rules: List[str]) -> bool:
    conds = []
    for r in rules:
        if r == "SMA20_gt_SMA50":
            conds.append(row.get("SMA20", np.nan) > row.get("SMA50", np.nan))
        elif r == "RSI_gt_55":
            conds.append(row.get("RSI14", 0) > 55)
        elif r == "MACD_hist_gt_0":
            conds.append(row.get("MACD_hist", -1e9) > 0)
        elif r == "Close_gt_BB_mid":
            conds.append(row.get("close", np.nan) > row.get("BB_mid", -1e9))
        elif r == "Bullish_Engulfing":
            conds.append(row.get("bullish_engulfing", 0) == 1)
        else:
            conds.append(True)
    return all(bool(x) for x in conds)

def generate_entry_signals(df: pd.DataFrame, cfg: Dict[str, Any]) -> pd.Series:
    rules = cfg["SIGNAL"]["rules"]
    return df.apply(lambda row: 1 if rule_conditions_satisfied(row, rules) else 0, axis=1).astype(int)

def make_ml_dataset(df: pd.DataFrame, horizon_days: int, thr_pct: float):
    # Label = 1 if max forward return within horizon_days >= thr_pct
    fut_max = df["close"].rolling(horizon_days).max().shift(-horizon_days+1)
    fwd_ret_pct = (fut_max / df["close"] - 1.0) * 100.0
    y = (fwd_ret_pct >= thr_pct).astype(int)

    X = df[[
        "SMA20","SMA50","EMA20","RSI14","MACD","MACD_signal","MACD_hist",
        "BB_upper","BB_mid","BB_lower","ATR14","ADX14","ATR_pct",
        "bullish_engulfing","bearish_engulfing","doji","hammer","hanging_man"
    ]].copy()
    X = X.replace([np.inf, -np.inf], np.nan).dropna()
    y = y.loc[X.index]
    return X, y

def train_ml_filter(df: pd.DataFrame, cfg: Dict[str, Any]):
    # Logistic Regression baseline with TimeSeriesSplit CV
    try:
        X, y = make_ml_dataset(df, cfg["SIGNAL"]["label_horizon_days"], cfg["SIGNAL"]["label_thr_pct"])
    except Exception as e:
        print("ML dataset error:", e)
        return None
    if len(X) < 200:
        print("Not enough data for ML filter; skipping.")
        return None
    tscv = TimeSeriesSplit(n_splits=5)
    best_model, best_auc = None, -1
    for tr, va in tscv.split(X):
        Xtr, Xv = X.iloc[tr], X.iloc[va]
        ytr, yv = y.iloc[tr], y.iloc[va]
        model = LogisticRegression(max_iter=200)
        model.fit(Xtr, ytr)
        prob = model.predict_proba(Xv)[:,1]
        auc = roc_auc_score(yv, prob)
        if auc > best_auc:
            best_auc, best_model = auc, model
    print(f"ML filter AUC (TS CV best): {best_auc:.3f}")
    return best_model

def apply_ml_filter(model, row: pd.Series) -> float:
    if model is None:
        return 1.0
    cols = ["SMA20","SMA50","EMA20","RSI14","MACD","MACD_signal","MACD_hist",
            "BB_upper","BB_mid","BB_lower","ATR14","ADX14","ATR_pct",
            "bullish_engulfing","bearish_engulfing","doji","hammer","hanging_man"]
    x = row[cols].values.reshape(1,-1)
    return float(model.predict_proba(x)[0,1])

print("✅ Stage 4 ready")


✅ Stage 4 ready



## Stage 5 — Risk Management & Position Sizing


In [7]:

def compute_costs(price: float, qty: int, cfg: Dict[str, Any], side: str) -> float:
    # Simplified fee model using placeholders. Adjust to your broker/exchange.
    costs = cfg["COSTS"]
    notional = price * qty
    brokerage = costs["brokerage_per_order"]
    slippage = (costs["slippage_bps"] / 10000.0) * notional
    stt = (costs["stt_equity_deliv_bps"] / 10000.0) * (notional if side.lower()=="sell" else 0.0)
    exch = (costs["exchange_txn_bps"] / 10000.0) * notional
    clr  = (costs["clearing_bps"] / 10000.0) * notional
    stamp = (costs["stamp_duty_bps_buy"] / 10000.0) * (notional if side.lower()=="buy" else 0.0)
    gst = (costs["gst_pct_on_brokerage"] / 100.0) * brokerage
    return brokerage + slippage + stt + exch + clr + stamp + gst

def position_size(entry_price: float, atr: float, cfg: Dict[str, Any], capital: float) -> int:
    risk_per_trade = cfg["RISK"]["risk_per_trade_pct"] / 100.0 * capital
    stop_distance = cfg["RISK"]["atr_stop_mult"] * atr
    if stop_distance <= 0:
        return 0
    qty = int(risk_per_trade // stop_distance)
    return max(qty, 0)

print("✅ Stage 5 ready")


✅ Stage 5 ready



## Stage 6 & 7 — Execution (Simulated), Monitoring & Exit


In [8]:

def backtest_symbol(df: pd.DataFrame, cfg: Dict[str, Any], ml_model=None, capital: float=0.0):
    df = df.copy()
    df["entry_signal"] = generate_entry_signals(df, cfg)

    atr_mult_sl = cfg["RISK"]["atr_stop_mult"]
    atr_mult_tsl = cfg["RISK"]["atr_trail_mult"]
    max_hold = cfg["RISK"]["max_hold_days"]
    ml_on = cfg["SIGNAL"]["ml_filter_enabled"]
    ml_thr = cfg["SIGNAL"]["ml_prob_threshold"]

    open_pos = None
    trades = []
    equity = []
    cash = capital
    pnl_running = 0.0

    for ts, row in df.iterrows():
        price = float(row["close"])
        atr = float(row.get("ATR14", np.nan))
        sma20, sma50 = row.get("SMA20", np.nan), row.get("SMA50", np.nan)

        if open_pos is not None:
            open_pos["day_count"] += 1
            open_pos["highest_close"] = max(open_pos["highest_close"], price)
            tsl = open_pos["highest_close"] - atr_mult_tsl * atr
            open_pos["stop"] = max(open_pos["stop"], tsl)

        def do_exit(reason: str):
            nonlocal open_pos, cash, pnl_running
            if open_pos is None:
                return
            exit_price = price
            qty = open_pos["qty"]
            gross = (exit_price - open_pos["entry_price"]) * qty
            sell_cost = compute_costs(exit_price, qty, cfg, side="sell")
            buy_cost  = open_pos["buy_cost"]
            net = gross - sell_cost - buy_cost
            pnl_running += net
            cash += (exit_price * qty - sell_cost)
            trades.append({
                "exit_time": ts, "exit_price": exit_price, "reason": reason,
                "entry_time": open_pos["entry_time"], "entry_price": open_pos["entry_price"],
                "qty": qty, "net_pnl": net, "gross_pnl": gross, "buy_cost": buy_cost, "sell_cost": sell_cost,
                "hold_days": open_pos["day_count"]
            })
            open_pos = None

        if open_pos is not None:
            if price <= open_pos["stop"]:
                do_exit("HARD/TSL_STOP")
            elif not (sma20 > sma50):
                do_exit("INDICATOR_EXIT")
            elif open_pos["day_count"] >= max_hold:
                do_exit("TIME_EXIT")

        if open_pos is None:
            if not passes_screen(df.loc[:ts], cfg):
                equity.append(pnl_running + cash)
                continue
            if row["entry_signal"] == 1:
                prob = 1.0
                if ml_on and (ml_model is not None):
                    try:
                        prob = apply_ml_filter(ml_model, row)
                    except Exception:
                        prob = 1.0
                if prob >= ml_thr:
                    qty = position_size(price, atr, cfg, cash if cash>0 else cfg["RISK"]["initial_capital"])
                    if qty > 0:
                        buy_cost = compute_costs(price, qty, cfg, side="buy")
                        cash -= (price * qty + buy_cost)
                        open_pos = {
                            "entry_time": ts,
                            "entry_price": price,
                            "qty": qty,
                            "stop": price - atr_mult_sl * atr,
                            "highest_close": price,
                            "day_count": 0,
                            "buy_cost": buy_cost
                        }
        equity.append(pnl_running + cash)

    equity_ser = pd.Series(equity, index=df.index)
    trades_df = pd.DataFrame(trades)
    return trades_df, equity_ser

print("✅ Stage 6 & 7 ready")


✅ Stage 6 & 7 ready



## Stage 8 — Performance & Feedback


In [9]:

def compute_max_drawdown(equity: pd.Series):
    roll_max = equity.cummax()
    drawdown = equity / roll_max - 1.0
    max_dd = drawdown.min()
    end_idx = drawdown.idxmin()
    start_idx = equity.loc[:end_idx].idxmax()
    return float(max_dd), start_idx, end_idx

def perf_metrics(trades: pd.DataFrame, equity: pd.Series, initial_capital: float):
    if equity.empty:
        return {"note": "No equity series."}
    total_return = (equity.iloc[-1] / initial_capital) - 1.0
    daily_returns = equity.pct_change().dropna()
    sharpe = (daily_returns.mean() / (daily_returns.std() + 1e-12)) * math.sqrt(252)
    downside = daily_returns[daily_returns < 0]
    sortino = (daily_returns.mean() / (downside.std() + 1e-12)) * math.sqrt(252)
    max_dd, dd_start, dd_end = compute_max_drawdown(equity)
    n = len(trades)
    wins = (trades["net_pnl"] > 0).sum() if n > 0 else 0
    win_rate = (wins / n * 100.0) if n > 0 else 0.0
    avg_pnl = trades["net_pnl"].mean() if n > 0 else 0.0
    gross_profit = trades.loc[trades["net_pnl"] > 0, "net_pnl"].sum() if n > 0 else 0.0
    gross_loss = trades.loc[trades["net_pnl"] < 0, "net_pnl"].sum() if n > 0 else 0.0
    profit_factor = (gross_profit / abs(gross_loss)) if gross_loss < 0 else np.nan
    days = max(1, (equity.index[-1] - equity.index[0]).days)
    years = days / 365.25
    cagr = (equity.iloc[-1] / initial_capital) ** (1/years) - 1.0 if years > 0 else np.nan
    return {
        "Total Return %": round(total_return*100, 2),
        "CAGR %": round(cagr*100, 2) if not np.isnan(cagr) else np.nan,
        "Sharpe": round(float(sharpe), 2),
        "Sortino": round(float(sortino), 2),
        "Max Drawdown %": round(max_dd*100, 2),
        "MaxDD Start": str(dd_start),
        "MaxDD End": str(dd_end),
        "Trades": int(n),
        "Win Rate %": round(win_rate, 2),
        "Avg PnL (INR)": round(avg_pnl, 2),
        "Profit Factor": round(float(profit_factor), 2) if profit_factor==profit_factor else np.nan
    }

def plot_equity(equity: pd.Series, title: str="Equity Curve"):
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(equity.index, equity.values)
    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel("Equity (INR)")
    plt.show()

print("✅ Stage 8 ready")


✅ Stage 8 ready



## Orchestration — Run the Full Pipeline


In [10]:

def run_pipeline(cfg: Dict[str, Any]):
    outdir = get_outdir()
    data_map = load_universe(cfg)

    feat_map = {}
    for t, df in data_map.items():
        try:
            feat_map[t] = add_features(df, cfg)
        except Exception as e:
            print("⚠️ Feature error", t, e)

    ml_model = None
    if cfg["SIGNAL"]["ml_filter_enabled"]:
        try:
            big = pd.concat([d.assign(ticker=k) for k, d in feat_map.items()]).sort_index()
            ml_model = train_ml_filter(big, cfg)
        except Exception as e:
            print("⚠️ ML training skipped:", e)

    initial_capital = cfg["RISK"]["initial_capital"]
    cash_per_symbol = initial_capital / max(1, len(feat_map))
    all_trades, eq_map = [], {}

    for t, df in feat_map.items():
        if not passes_screen(df, cfg):
            print("Skip", t, "failed screen.")
            continue
        try:
            tr, eq = backtest_symbol(df, cfg, ml_model=ml_model, capital=cash_per_symbol)
            if not tr.empty:
                tr["ticker"] = t
                all_trades.append(tr)
            eq_map[t] = eq
        except Exception as e:
            print("⚠️ Backtest error", t, e)

    if len(all_trades) == 0:
        print("No trades generated.")
        return {"trades": pd.DataFrame(), "metrics": {}}

    trades_df = pd.concat(all_trades).sort_values("exit_time")
    eq_df = pd.concat(eq_map, axis=1).fillna(method="ffill").fillna(method="bfill")
    port_equity = eq_df.sum(axis=1)

    metrics = perf_metrics(trades_df, port_equity, initial_capital)
    print("Summary Metrics:", json.dumps(metrics, indent=2))

    if cfg["OUTPUTS"]["save_csv"]:
        trades_df.to_csv(outdir / "trades.csv", index=False)
        print("Saved trades to", outdir / "trades.csv")

    if cfg["OUTPUTS"]["save_equity_plot"]:
        plot_equity(port_equity, title="Portfolio Equity Curve")

    return {"trades": trades_df, "equity": port_equity, "metrics": metrics, "outdir": outdir}

print("✅ Orchestration ready")


✅ Orchestration ready



## (Optional) Telegram Alerts — Stub


In [11]:

def send_telegram(message: str, bot_token: str, chat_id: str):
    # Minimal Telegram sender stub for live mode integration.
    try:
        import requests
        url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
        payload = {"chat_id": chat_id, "text": message}
        # r = requests.post(url, data=payload, timeout=10)
        print("[stub] Would send Telegram message:", message[:120], "...")
    except Exception as e:
        print("Telegram send failed:", e)

print("✅ Telegram stub ready")


✅ Telegram stub ready



## Quick Start — Run the Pipeline


In [12]:

# results = run_pipeline(CONFIG)
# results["trades"].head()
